記述統計へのDP適用

In [1]:
!pip install ja-ginza

DEPRECATION: Loading egg at /home/jun/anaconda3/lib/python3.11/site-packages/python_dlshogi2-0.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 56.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 92.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 29.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
data_path = './data/reviews_with_sentiment.csv'

df = pd.read_csv(data_path)
df

,review,sentiment
0,出張でお世話になりました。,neutral
1,朝食は普通でした。,neutral
2,また是非行きたいです。,positive
3,また利用したいと思えるホテルでした。,positive
4,駅から近くて便利でした。,positive
...,...,...
5548,スクランブルエッグにとても大きな殻が入っていてざりざりして気持ちが悪く未だに思い出します。,negative
5549,何組か食事してましたが、明らかにレストランではないスペース。,negative
5550,静かで居心地も良いしまた行きたい宿になりましたが、一点だけ残念だったのが夕食の海鮮舟盛りです。,neutral
5551,チェックインして、部屋に入るとベッドルームの照明の位置もわかりにくく、大変な思いをしました。,negative


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import itertools
from typing import List, Tuple
import numpy as np

# データの読み込み
data_path = './data/reviews_with_sentiment.csv'
df = pd.read_csv(data_path)

# spaCyの日本語モデルのロード
nlp = spacy.load('ja_ginza')

# 抽出する品詞の指定
POS = ['ADJ', 'ADV', 'INTJ', 'PROPN', 'NOUN', 'VERB']
MAX_TERMS_IN_DOC = 5
NGRAM = 1
MAX_DF = 1.0
MIN_DF = 0.0
NUM_VOCAB = 10000
TOP_K = 20

def flatten(*lists) -> list:
    res = []
    for l in list(itertools.chain.from_iterable(lists)):
        for e in l:
            res.append(e)
    return res

def remove_duplicates(l: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
    d = {}
    for e in l:
        d[e[0]] = e[1]
    return list(d.items())

# 形態素解析
df["doc"] = [nlp(review) for review in df["review"]]

bows = {}
cvs = {}

for sentiment in df["sentiment"].unique():
    tokens = []
    for doc in df[df["sentiment"] == sentiment]["doc"]:
        similarities = [(token.similarity(doc), token.lemma_) for token in doc if token.pos_ in POS]
        similarities = remove_duplicates(similarities)
        similarities = sorted(similarities, key=lambda sim: sim[1], reverse=True)[:MAX_TERMS_IN_DOC]
        tokens.append([similarity[1] for similarity in similarities])
    
    cv = CountVectorizer(ngram_range=(1, NGRAM), max_df=MAX_DF, min_df=MIN_DF, max_features=NUM_VOCAB)
    bows[sentiment] = cv.fit_transform(flatten(tokens)).toarray()
    cvs[sentiment] = cv

# 上位単語の頻度を計算
vocabs = {}
term_frequencies = {}

for sentiment in df["sentiment"].unique():
    bow = bows[sentiment]
    cv = cvs[sentiment]
    
    vocab = cv.vocabulary_
    term_frequency = np.sum(bow, axis=0)
    vocabs[sentiment] = vocab
    term_frequencies[sentiment] = term_frequency
    
    indices_topk = np.argsort(term_frequency)[::-1][:TOP_K]
    bow_topk = np.take(bow, indices_topk, axis=1)
    reverse_vocab = {vocab[k]: k for k in vocab.keys()}
    words = [reverse_vocab[i] for i in indices_topk]
    
    print(sentiment, ":")
    for w, c in zip(words, term_frequency[indices_topk]):
        print(w, ":", c)


/tmp/ipykernel_218214/2839339426.py:46: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarities = [(token.similarity(doc), token.lemma_) for token in doc if token.pos_ in POS]


neutral :
利用 : 216
部屋 : 93
旅行 : 67
ある : 60
思う : 59
朝食 : 50
風呂 : 47
普通 : 45
宿泊 : 43
なる : 43
初めて : 34
ホテル : 32
いる : 30
良い : 27
感じ : 27
家族 : 25
出張 : 25
予約 : 23
する : 22
夕食 : 21
positive :
良い : 463
部屋 : 315
美味しい : 206
利用 : 198
満足 : 193
朝食 : 181
思う : 175
ある : 144
風呂 : 141
最高 : 140
ホテル : 124
対応 : 106
いる : 102
また : 99
温泉 : 96
清潔 : 90
よい : 90
接客 : 87
綺麗 : 84
気持ち : 80
negative :
残念 : 99
部屋 : 97
少し : 48
風呂 : 42
朝食 : 39
狭い : 36
思う : 34
ある : 33
ない : 27
良い : 24
なる : 22
悪い : 19
欲しい : 16
する : 16
露天風呂 : 16
古い : 16
掃除 : 15
食べる : 15
食事 : 15
改善 : 14


In [ ]:
!pip install ja-ginza
!pip install spacy
!pip install sklearn
!pip install python-dp

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import itertools
from typing import List, Tuple

# データの読み込み
data_path = './data/reviews_with_sentiment.csv'
df = pd.read_csv(data_path)

# spaCyの日本語モデルのロード
nlp = spacy.load('ja_ginza')

# 抽出する品詞の指定
POS = ['ADJ', 'ADV', 'INTJ', 'PROPN', 'NOUN', 'VERB']
MAX_TERMS_IN_DOC = 5
NGRAM = 1
MAX_DF = 1.0
MIN_DF = 0.0
NUM_VOCAB = 10000

def flatten(*lists) -> list:
    res = []
    for l in list(itertools.chain.from_iterable(lists)):
        for e in l:
            res.append(e)
    return res

def remove_duplicates(l: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
    d = {}
    for e in l:
        d[e[0]] = e[1]
    return list(d.items())

# 形態素解析
df["doc"] = [nlp(review) for review in df["review"]]
